In [ ]:
# TASK 4.2 NLP - HRITHIK NAMBAR
#REDDIT FLAIR CLASSIFICATION

In [ ]:
#importing the necessary libraries

In [2]:
import pandas as pd 
import numpy as np

%matplotlib inline
import os, sys
import re
import string
import pathlib
import random
from collections import Counter, OrderedDict
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
from tqdm import tqdm, tqdm_notebook, tnrange
tqdm.pandas(desc='Progress')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torchtext
from torchtext import data
from torchtext import vocab
from torchtext import datasets
from torchtext.vocab import Vectors, GloVe
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.metrics import accuracy_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

import warnings
warnings.filterwarnings('ignore')


In [4]:
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
# Analysing the data

In [6]:
train = pd.read_csv("train.txt")
train.head(10)

,Title,Post,Flair
0,netflix the family has been an amazing watch d...,netflixs new series the family is about a secr...,10
1,all results are out is iiitm gwalior it branch...,the internet seems to think so average package...,0
2,which are the things you always buy made in india,you can include the reason for your preference...,0
3,weekly coders hackers amp all tech related thr...,last week issue all threads every week on frid...,11
4,what are some good unknown companies to work a...,there are similar posts on other subreddits bu...,0
5,is tinder worth my time,i heard a lot about it from my friends and col...,0
6,is there a progressive political party in indi...,i am an american i vote progressive democrat i...,10
7,rindia father heavily depressed situation is r...,father took a big loan couldn pay amount kept ...,7
8,what to do when more than mrp was charged in f...,hello i had ordered a perfume for which costed...,0
9,app for expense tracking budgeting financial a...,need recommendations for an app which is both ...,1


In [7]:
train["Flair"].max()

14

In [8]:
label_to_id={"AskIndia": 0, "Business/Finance": 1, "CAA-NRC": 2, "Coronavirus": 3, "Demonetization": 4, "Entertainment": 5, "Food": 6, "Non-Political": 7, "Photography": 8, "Policy/Economy": 9, "Politics": 10, "Scheduled": 11, "Science/Technology": 12, "Sports": 13, "[R]eddiquette": 14}
id_to_label={v: k for k, v in label_to_id.items()}
id_to_label

{0: 'AskIndia',
 1: 'Business/Finance',
 2: 'CAA-NRC',
 3: 'Coronavirus',
 4: 'Demonetization',
 5: 'Entertainment',
 6: 'Food',
 7: 'Non-Political',
 8: 'Photography',
 9: 'Policy/Economy',
 10: 'Politics',
 11: 'Scheduled',
 12: 'Science/Technology',
 13: 'Sports',
 14: '[R]eddiquette'}

In [9]:
flairs=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14]
#flairs = [‘AskIndia’, ‘Business/Finance’, ‘CAA-NRC’, ‘Coronavirus’, ‘Demonetization’, ‘Entertainment’, ‘Food’, ‘Non-Political’, ‘Photography’, ‘Policy/Economy’, ‘Politics’, ‘Scheduled’, ‘Science/Technology’, ‘Sports’, ‘[R]eddiquette’]

In [10]:
for flair in id_to_label:
    print (flair,': ' , len(train[train['Flair'] == flair]))

0 :  16616
1 :  811
2 :  76
3 :  793
4 :  365
5 :  172
6 :  293
7 :  8118
8 :  96
9 :  1046
10 :  4847
11 :  580
12 :  961
13 :  256
14 :  1517


In [11]:
val = pd.read_csv("val.txt")
for flair in flairs:
    print (flair,': ' , len(val[val['Flair'] == flair]))

0 :  4092
1 :  200
2 :  19
3 :  196
4 :  90
5 :  42
6 :  72
7 :  1999
8 :  24
9 :  257
10 :  1194
11 :  143
12 :  236
13 :  63
14 :  374


In [12]:
#Here we replace removed, nan or deleted text in selftext with an empty character

In [13]:
train['Post'] = train['Post'].astype(str)
val['Post'] = val['Post'].astype(str)

empties = ['nan', '[deleted]', '[removed]']
for empty in empties:
  print(empty, len(train[train['Post'] == empty]))

nan 238
[deleted] 0
[removed] 0


In [14]:
train['Post'] = train['Post'].apply(lambda x: '' if x in empties else x)
val['Post'] = val['Post'].apply(lambda x: '' if x in empties else x)

In [15]:
empties = ['nan', '[deleted]', '[removed]']
for empty in empties:
  print(empty, len(train[train['Post'] == empty]))

nan 0
[deleted] 0
[removed] 0


In [16]:
#let us merge the title and post since they describe the same thing

In [17]:
train['Title'] = train['Title'] + ' ' + train['Post']
val['Title'] = val['Title'] + ' ' + val['Post']

In [18]:
import unicodedata
import re
nlp = spacy.load('en',disable=['parser', 'tagger', 'ner'])

def tokenizer(s): 
    return [w.text.lower() for w in nlp(s)]

In [19]:
cols = ['Post']
train, val = train.drop(cols, axis=1), val.drop(cols, axis=1)

In [20]:
train.head()

,Title,Flair
0,netflix the family has been an amazing watch d...,10
1,all results are out is iiitm gwalior it branch...,0
2,which are the things you always buy made in in...,0
3,weekly coders hackers amp all tech related thr...,11
4,what are some good unknown companies to work a...,0


In [21]:
train.to_csv('train.csv', index=False);val.to_csv('val.csv', index=False)

In [22]:
label_field = data.Field(sequential=False, use_vocab=False, pad_token=None, unk_token=None)
text_field = data.Field(sequential=True, tokenize=tokenizer, include_lengths=True, use_vocab=True)

train_val_fields = [
     ('Title', text_field),
    ('flairs', label_field)
   
]

trainds, valds = data.TabularDataset.splits(path='./', format='csv', train='train.csv', 
                validation='val.csv', fields=train_val_fields, skip_header=True)

In [23]:
#pre-processing complete.

In [24]:
pretrained_GloVe_sizes = [50, 100, 200, 300]

def load_pretrained_vectors(dim):
    if dim in pretrained_GloVe_sizes:
        # for other pretrained vectors. 6B used here
        name = 'glove.{}.{}d'.format('6B', str(dim))
        return name
    return None

vectors = load_pretrained_vectors(300)

In [25]:
text_field.build_vocab(trainds, valds, max_size=100000, vectors=vectors)
label_field.build_vocab(trainds)

In [26]:
text_field.vocab.vectors.shape

torch.Size([100002, 300])

In [27]:
train.head(1)

,Title,Flair
0,netflix the family has been an amazing watch d...,10


In [28]:
print (text_field.vocab.freqs.most_common(10))
print (label_field.vocab.freqs.most_common(10))

[('the', 201988), ('to', 157891), ('i', 143063), ('and', 136361), ('a', 115617), ('of', 110392), ('in', 98378), ('is', 79377), ('for', 62627), ('it', 60650)]
[('0', 16616), ('7', 8118), ('10', 4847), ('14', 1517), ('9', 1046), ('12', 961), ('1', 811), ('3', 793), ('11', 580), ('4', 365)]


In [29]:
#Here we build RNN/LSTM based models for flair classification task.  
#we input the concatenation of title and body of the post and try to predict flair using a softmax classifier.

In [30]:
vocab_size = len(text_field.vocab)
embedding_dim = 300
n_hidden = 128
n_out = 11

In [31]:
#First we have a LSTM or GRU encoder for the embedded input sequence
#Here we have a context vector(attention mechanism) conditioned over the hidden states of the GRU or LSTM
#First we have a LSTM or GRU encoder for the embedded input sequence

In [32]:
RNNS = ['LSTM', 'GRU']

class Encoder(nn.Module):
  
  def __init__(self, embedding_dim, hidden_dim, nlayers=1, dropout=0.,bidirectional=True, rnn_type='GRU'):
    super(Encoder, self).__init__()
    self.bidirectional = bidirectional
    assert rnn_type in RNNS, 'Use one of the following: {}'.format(str(RNNS))
    rnn_cell = getattr(nn, rnn_type)
    self.rnn = rnn_cell(embedding_dim, hidden_dim, nlayers, 
                        dropout=dropout, bidirectional=bidirectional)
  
  def forward(self, input, hidden=None):
    return self.rnn(input, hidden)

In [33]:
#Attention layer
#We then perform a Scaled dot-product,self-attention with the encoder outputs as keys and values and the hidden state as the query.

In [34]:
class Attention(nn.Module):
  
  def __init__(self, query_dim, key_dim, value_dim):
    super(Attention, self).__init__()
    self.scale = 1. / math.sqrt(query_dim)
  

  def forward(self, query, keys, values):
    query = query.unsqueeze(1)
    keys = keys.transpose(0,1).transpose(1,2)
    energy = torch.bmm(query, keys) 
    energy = F.softmax(energy.mul_(self.scale), dim=2) 

    values = values.transpose(0,1)
    linear_combination = torch.bmm(energy, values).squeeze(1) 
    return energy, linear_combination

In [35]:
# Add a softmax classifier on top of the attention outputs.

In [36]:
class Classifier(nn.Module):
  
 def __init__(self, embedding, encoder, attention, hidden_dim, num_classes):
    super(Classifier, self).__init__()
    self.embedding = embedding
    self.encoder = encoder
    self.attention = attention
    self.decoder = nn.Linear(hidden_dim, num_classes)

    size = 0
    for p in self.parameters():
        size += p.nelement()
    print('Total param size: {}'.format(size))

  

 def forward(self, input):
    outputs, hidden = self.encoder(self.embedding(input))
    if isinstance(hidden, tuple):
        hidden = hidden[1]

    if self.encoder.bidirectional:
        hidden = torch.cat([hidden[-1], hidden[-2]], dim=1)
    else:
        hidden = hidden[-1]

    energy, linear_combination = self.attention(hidden, outputs, outputs) 
    logits = self.decoder(linear_combination)
    return logits, energy

In [37]:
def seed_everything(seed, cuda=False):
  # Set the random seed manually for reproducibility.
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def acc(accuracy, logits, y):
    _, max_ind = torch.max(logits, 1)
    equal = torch.eq(max_ind, y)
    correct = int(torch.sum(equal))
    return accuracy + correct

In [38]:
# Training: Takes in the model, optimizer(Adam), dataloader, loss function

In [39]:
def train(epoch, model, trainiter, valiter, optimizer, criterion):
    model.train()
    train_accuracy = 0
    t = time.time()
    train_total_loss = 0
    for batch_num, batch in enumerate(trainiter):
        model.zero_grad()
        x, lens = batch.Title
        y = batch.flairs
        logits, _ = model(x)
        loss = criterion(logits.view(-1, 15), y)
        train_total_loss += float(loss)
        train_accuracy = acc(train_accuracy, logits, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.25)
        optimizer.step()
    
    model.eval()
    val_accuracy = 0
    t = time.time()
    val_total_loss = 0
    with torch.no_grad():
        for batch_num, batch in enumerate(valiter):
            x, lens = batch.Title
            y = batch.flairs
            logits, _ = model(x)
            val_total_loss += float(criterion(logits.view(-1, 15), y))
            val_accuracy = acc(val_accuracy, logits, y)    
        t = time.time()

    train_lo = (train_total_loss / len(trainiter))
    train_acc = (train_accuracy / len(trainiter.dataset))
    val_lo = (val_total_loss / len(valiter))
    val_acc = (val_accuracy / len(valiter.dataset))
  
    print(f'Epoch {epoch}: train_loss: {train_lo:.4f} train_acc: {train_acc:.4f} | val_loss: {val_lo:.4f} val_acc: {val_acc:.4f}')
    return model

In [40]:
cuda = torch.cuda.is_available()
device = torch.device("cpu") if not cuda else torch.device("cuda:0")
seed_everything(seed=1337, cuda=cuda)

In [41]:
batch_size = 128
traindl, valdl = data.BucketIterator.splits(datasets=(trainds, valds), 
                                            batch_size=batch_size, 
                                            sort_key=lambda x: len(x.Title), 
                                            device=device, 
                                            sort_within_batch=True, 
                                            repeat=False)

In [42]:
train_iter, val_iter = traindl, valdl
TEXT, LABEL = text_field, label_field

In [43]:
print("[Corpus]: train: {}, test: {}, vocab: {}, labels: {}".format(
            len(train_iter.dataset), len(val_iter.dataset), len(TEXT.vocab), len(LABEL.vocab)))

ntokens, nlabels = len(TEXT.vocab), len(LABEL.vocab)

[Corpus]: train: 36547, test: 9001, vocab: 100002, labels: 15


In [44]:
import math

#Hyperparameters
emsize = 300
hidden = 64
nlayers = 2
lr = 1e-3
clip = 0.25
epochs = 3
drop = 0.6
model = 'GRU'
bi = True
criterion = nn.CrossEntropyLoss()

In [45]:
embedding = nn.Embedding(ntokens, emsize, padding_idx=1, max_norm=1)
if vectors: 
    embedding.weight.data.copy_(TEXT.vocab.vectors)
encoder = Encoder(emsize, hidden, nlayers=nlayers, 
                  dropout=drop, bidirectional=bi, rnn_type=model)

attention_dim = hidden if not bi else 2*hidden
attention = Attention(attention_dim, attention_dim, attention_dim)

model = Classifier(embedding, encoder, attention, attention_dim, nlabels)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr, amsgrad=True)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

Total param size: 30217575


Classifier(
  (embedding): Embedding(100002, 300, padding_idx=1, max_norm=1)
  (encoder): Encoder(
    (rnn): GRU(300, 64, num_layers=2, dropout=0.6, bidirectional=True)
  )
  (attention): Attention()
  (decoder): Linear(in_features=128, out_features=15, bias=True)
)

In [46]:
for epoch in range(1, epochs + 1):
    m = train(epoch, model, train_iter, val_iter, optimizer, criterion)

Epoch 1: train_loss: 1.4976 train_acc: 0.5297 | val_loss: 1.2788 val_acc: 0.6088
Epoch 2: train_loss: 1.0776 train_acc: 0.6625 | val_loss: 1.2670 val_acc: 0.5994
Epoch 3: train_loss: 0.7418 train_acc: 0.7659 | val_loss: 1.4815 val_acc: 0.5733


In [ ]:
# Accuracy of 57.33% is achieved.